<a href="https://colab.research.google.com/github/ShinAsakawa/2015corona/blob/master/notebooks/2021_0531ccap_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021年5月版 word2vec デモ

- date: 2021_0510
- filename: 2021_0530ccap_word2vec.pynb
- author: 浅川伸一
- 概要:

In [ ]:
#colab 上で MeCab をインストール際に，次行をコメントアウト
!pip install mecab-python3==0.996.3
import MeCab

In [ ]:
!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb" -O 2021_05jawiki_hid128_win10_neg10_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M" -O 2021_05jawiki_hid128_win10_neg10_sgns.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s" -O 2021_05jawiki_hid200_win20_neg20_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7" -O 2021_05jawiki_hid200_win20_neg20_sgns.bin.gz && rm -rf /tmp/cookies.txt

In [ ]:
%time
import numpy as np
# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

# word2vec データ処理のため gensim を使う
import os
import sys
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

print('# word2vec データの読み込み')
print('# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...', end="")
# ファイルの所在に応じて変更してください
w2v_base = '.'
w2v_file='2021_05jawiki_hid128_win10_neg10_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid128_win10_neg10_sgns.bin.gz'
#w2v_file='2021_05jawiki_hid200_win20_neg20_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid200_win20_neg20_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file, 
                                                encoding='utf-8', 
                                                unicode_errors='replace',
                                                binary=True) 

w2v = asakawa_w2v
print('done')

In [ ]:
#以下は操作例です。

word = '鬼滅の刃' #入力単語
topn = 10.       #類似度が高い順に何語出力するか
word = input('単語を入力してください')
w2v.most_similar(word)

In [ ]:
w2v.most_similar(positive=('王','女'), negative=('男'))

In [ ]:
w2v.most_similar(positive=('早稲田大学','関西'), negative=('関東'))

In [ ]:
w2v.most_similar(positive=('ジャイアンツ','アメリカ合衆国'), negative=('日本'))

In [ ]:
w2v.most_similar(positive=('鬼滅の刃'))

In [ ]:
w2v.vector_size

200